In [1]:
import argparse
import os
import sys
import yaml
from shutil import copyfile
from datetime import datetime
import cv2
import torch
import torch.nn as nn
from torchvision import transforms
import torchvision.models as models
from torch.utils.tensorboard import SummaryWriter
from visualize_IG import *
from utils_IG import *
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image
from smooth_grad.gradients import VanillaGrad, SmoothGrad, GuidedBackpropGrad, GuidedBackpropSmoothGrad
from smooth_grad.image_utils import preprocess_image, save_as_gray_image
from smooth_grad.labels import IMAGENET_LABELS


/home/kaiser17/anaconda3/envs/patra2/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
sys.path.insert(1, os.path.join(sys.path[0], '../'))
from cartoonX import CartoonX 
from pixelRDE import PixelRDE
from utils_IG import calculate_outputs_and_gradients, generate_entrie_images
from Integrated_grad import random_baseline_integrated_gradients

In [3]:
LABEL_LIST = tuple(open(os.path.join(sys.path[0], "imagenet_labels.txt")).read().split('\n'))
LABEL_LIST = [x.replace('{',"").replace('\'',"").replace(',',"").replace('-'," ").replace('_'," ") for x in LABEL_LIST]

In [4]:
files = os.listdir('random_images/')
imgdir='random_images'

In [6]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

# Get classifier to explain
model = models.mobilenet_v3_small(pretrained=True).eval().to(device)

In [7]:
with open(os.path.join(sys.path[0], "hparams.yaml")) as f:
    HPARAMS_CARTOONX = yaml.load(f, Loader=yaml.FullLoader)["CartoonX"]

with open(os.path.join(sys.path[0], "hparams.yaml")) as f:
    HPARAMS_PIXEL_RDE = yaml.load(f, Loader=yaml.FullLoader)["PixelRDE"]

# Initialize wavelet RDE and pixel RDE
cartoonX = CartoonX(model=model, device=device, **HPARAMS_CARTOONX)
pixelRDE = PixelRDE(model=model, device=device, **HPARAMS_PIXEL_RDE)


In [8]:
def k_largest_index_argsort(a, k):
    idx = np.argsort(a.ravel())[:-k-1:-1]
    return np.column_stack(np.unravel_index(idx, a.shape))

def createmaskedimage(img,grad,percent,noise):
    #grad:L X B
    num_pixels=int(grad.shape[0]*grad.shape[1]*percent)
    mask=np.zeros((grad.shape[0],grad.shape[1]))
    topk_idx=k_largest_index_argsort(grad,num_pixels)
    mask[topk_idx[:,0],topk_idx[:,1]]=1
    mask=torch.tensor(mask).unsqueeze(0).unsqueeze(0).to(device)#1 X 1 X L X B
    noise=torch.randn(img.shape)
    new_img=img*mask+torch.clip(noise.to(device)*(1-mask),0,1)
    return(mask,new_img)

def save_as_gray_image(img, filename, percentile=99):#smoothgrad
    img_2d = np.sum(img, axis=0)
    span = abs(np.percentile(img_2d, percentile))
    vmin = -span
    vmax = span
    img_2d = np.clip((img_2d - vmin) / (vmax - vmin), -1, 1)
    cv2.imwrite(filename, img_2d * 255)

In [9]:
percent_list=list(np.array(list(range(0,101)))/100)
full_array=torch.zeros((len(files),len(percent_list)))
full_array_smooth=torch.zeros((len(files),len(percent_list)))
full_array_cartoon=torch.zeros((len(files),len(percent_list)))
full_array_randomwavelet=torch.zeros((len(files),len(percent_list)))

## Integrated Gradients

In [ ]:
#Integrated Gradients

for i,fname in enumerate(files):
    print(f"Processing file: {fname}")
    # Get image and transform to tensor    
    img = cv2.imread(os.path.join(imgdir, fname))
    img = cv2.resize(img, (256, 256))
    img = img.astype(np.float32) 
    img = img[:, :, (2, 1, 0)]
    # calculate the gradient and the label index
    gradients, label_index = calculate_outputs_and_gradients([img], model, None, True)
    gradients = np.transpose(gradients[0], (1, 2, 0))
    attributions = random_baseline_integrated_gradients(img, model, label_index, calculate_outputs_and_gradients, \
                                                        steps=100, num_random_trials=10, cuda=True)

    
    input_img = pre_processing(img, True)

    with torch.no_grad():
        for j,percent in enumerate(percent_list):
            noise=torch.randn(input_img.shape)
            mask_,masked_img=createmaskedimage(input_img,attributions.mean(2),percent,noise)
            if torch.argmax(model(input_img),1)==torch.argmax(model(masked_img.float()),1):

                full_array[i,j]=1
    
    
    
    
#     break

In [ ]:
np.save('ACCIntegG.npy', full_array)
# xxx=np.load('ACCIntegG.npy')
# plt.plot(xxx.mean(0))

## SmoothGrad

In [ ]:
#SmoothGrad
for i,fname in enumerate(files):
    print(f"Processing file: {fname}")
    # Get image and transform to tensor    
    img = cv2.imread(os.path.join(imgdir, fname))
    img = cv2.resize(img, (256, 256))
    img = img.astype(np.float32) 
    img = img[:, :, (2, 1, 0)]
    input_img = pre_processing(img, True)
    
    output = model(input_img)
    pred_index = np.argmax(output.data.cpu().numpy())
    smooth_grad = SmoothGrad(
        pretrained_model=model,
        cuda=True,
        n_samples=1000,
        magnitude=True)
    smooth_saliency = smooth_grad(input_img, index=None)

    input_img = pre_processing(img, True)  
    with torch.no_grad():
        for j,percent in enumerate(percent_list):
            noise=torch.randn(input_img.shape)
            mask_,masked_img=createmaskedimage(input_img,smooth_saliency.sum(0),percent,noise)
            if torch.argmax(model(input_img),1)==torch.argmax(model(masked_img.float()),1):

                full_array_smooth[i,j]=1    


In [ ]:
np.save('ACCSmoothG.npy', full_array_smooth)
# xxx=np.load('ACCSmoothG.npy')
# plt.plot(xxx.mean(0))

## Guided Backprop

In [ ]:
#Guided BackProp
full_array_GBP=torch.zeros((len(files),len(percent_list)))
for i,fname in enumerate(files):
    print(f"Processing file: {fname}")
    # Get image and transform to tensor    
    img = cv2.imread(os.path.join(imgdir, fname))
    img = cv2.resize(img, (256, 256))
    img = img.astype(np.float32) 
    img = img[:, :, (2, 1, 0)]
    input_img = pre_processing(img, True)
    
    output = model(input_img)
    pred_index = np.argmax(output.data.cpu().numpy())

    guided_grad = GuidedBackpropGrad(
        pretrained_model=model, cuda=True)
    
    guided_saliency = guided_grad(input_img, index=None)
    input_img = pre_processing(img, True)   
    with torch.no_grad():
        for j,percent in enumerate(percent_list):
            noise=torch.randn(input_img.shape)
            mask_,masked_img=createmaskedimage(input_img,guided_saliency.sum(0),percent,noise)

            if torch.argmax(model(input_img),1)==torch.argmax(model(masked_img.float()),1):

                full_array_GBP[i,j]=1        
    


In [ ]:
np.save('ACCGuidedBP.npy', full_array_GBP)
xxx=np.load('ACCGuidedBP.npy')
plt.plot(xxx.mean(0))

In [15]:
def marker(mask,percent):
    if percent==0:
        return 99999999
    elif percent==1:
#         print("right")
        return -9999999
    mask_open=[mask[0].reshape(-1)]
    for i in range(len(mask[1])):
        mask_open.append(mask[1][i].reshape(-1))
    final_mask_open=torch.cat(mask_open,0).detach().cpu().numpy()
    final_mask_open=final_mask_open.reshape(-1,1) # 60000 X 1
    noise_symmet=torch.randn(final_mask_open.shape).numpy()*0.000000001
    topk_idx=k_largest_index_argsort(noise_symmet+final_mask_open,int(final_mask_open.shape[0]*percent))
    return(final_mask_open[topk_idx[-1][0],topk_idx[-1][1]])

def createmaskedwavelt(wavelet,grad_mask,mark):
    

#     print("look-",wavelet.shape,mask.shape)
    noise=(torch.randn(wavelet.shape).to(device)*(torch.std(wavelet)).to(device))+torch.mean(wavelet).to(device)
    new_wavelet=wavelet*(grad_mask>=mark)+noise.to(device)*((grad_mask<mark))
    return(mask,new_wavelet.float())



## CartoonX

In [ ]:
# CartoonX
for i,fname in enumerate(files):
    print(f"Processing file: {fname}")
    x = Image.open(os.path.join(imgdir, fname))
    x = transforms.ToTensor()(x)
    x = transforms.Resize(size=(256,256))(x)
    x = x.to(device).unsqueeze(0)
    input_img=x.detach()
    # classifiy image
    output = model(x)
    pred = nn.Softmax(dim=1)(output).max(1)[1].item() 

    # set CartoonX hyperparameters
    HPARAMS ={"wave": "db3", "mode": "zero", "J": 5,
              "l1lambda": 10, "step_size": 1e-3,
              "num_steps": 100,  "batch_size": 16,
              "distortion_measure": "label"} 

    # get Cartoon RDE 
    cartoonX = CartoonX(model=model, device=device,return_mask=True, **HPARAMS)
    explanation,mask = cartoonX(x, pred)
    
    with torch.no_grad():
        for j,percent in enumerate(percent_list):
            yl, yh = cartoonX.forward_dwt(x) 
            markz=marker(mask,percent)
            _,obf_yl = createmaskedwavelt(yl,mask[0],markz)
            obf_yh = []
            for count, y in enumerate(yh):

                obf_yh.append(createmaskedwavelt(yh[count],mask[1][count],markz)[1])
            z = cartoonX.inverse_dwt((obf_yl,obf_yh))
            masked_img = z.clamp(0,1) 
    
            if torch.argmax(model(input_img),1)==torch.argmax(model(masked_img.float()),1):

                full_array_cartoon[i,j]=1       
    
    
    

In [ ]:
np.save('ACcartoon.npy', full_array_cartoon)
# xxx=np.load('ACcartoon.npy')
# plt.plot(xxx.mean(0))


## Random Wavelet

In [ ]:
#Random wavelet



for i,fname in enumerate(files):
    print(f"Processing file: {fname}")
    x = Image.open(os.path.join(imgdir, fname))
    x = transforms.ToTensor()(x)
    x = transforms.Resize(size=(256,256))(x)
    x = x.to(device).unsqueeze(0)
    input_img=x
    # classifiy image
    output = model(x)
    pred = nn.Softmax(dim=1)(output).max(1)[1].item() 

    # set CartoonX hyperparameters
    HPARAMS ={"wave": "db3", "mode": "zero", "J": 5,
              "l1lambda": 20, "step_size": 1e-3,
              "num_steps": 100,  "batch_size": 16,
              "distortion_measure": "label"} 

    # get Cartoon RDE 
    cartoonX = CartoonX(model=model, device=device,return_mask=True, **HPARAMS)
    explanation,mask = cartoonX(x, pred)
    
    with torch.no_grad():
        mask[0]=torch.randn(mask[0].shape).to(device)
        for dd in range(len(mask[1])):
            mask[1][dd]=torch.randn(mask[1][dd].shape).to(device)
        for j,percent in enumerate(percent_list):
            yl, yh = cartoonX.forward_dwt(x) 
            markz=marker(mask,percent)
            _,obf_yl = createmaskedwavelt(yl,mask[0],markz)
            obf_yh = []
            for count, y in enumerate(yh):

                obf_yh.append(createmaskedwavelt(yh[count],mask[1][count],markz)[1])
            z = cartoonX.inverse_dwt((obf_yl,obf_yh))
            masked_img = z.clamp(0,1) 
            if torch.argmax(model(input_img),1)==torch.argmax(model(masked_img.float()),1):

                full_array_randomwavelet[i,j]=1        
    

In [ ]:
np.save('ACrandomwavelet.npy', full_array_randomwavelet)
# xxx=np.load('ACrandomwavelet.npy')
# plt.plot(xxx.mean(0))


## Random Pixel

In [ ]:
#Random Pixel
full_array_randompixel=torch.zeros((len(files),len(percent_list)))


for i,fname in enumerate(files):
    print(f"Processing file: {fname}")
    # Get image and transform to tensor    
    img = cv2.imread(os.path.join(imgdir, fname))
    img = cv2.resize(img, (256, 256))
    img = img.astype(np.float32) 
    img = img[:, :, (2, 1, 0)]
    input_img = pre_processing(img, True)
    
    with torch.no_grad():
        for j,percent in enumerate(percent_list):
            noise=torch.randn(input_img.shape)
            mask_,masked_img=createmaskedimage(input_img,np.random.randn(256,256),percent,noise)
            
            if torch.argmax(model(input_img),1)==torch.argmax(model(masked_img.float()),1):

                full_array_randompixel[i,j]=1               
            
#             full_array_randompixel[i,j]=((F.softmax(model(input_img),1)-F.softmax(model(masked_img.float()),1))**2).sum()
    


In [ ]:
np.save('ACrandompixel.npy', full_array_randompixel)
# xxx=np.load('ACrandompixel.npy')
# plt.plot(xxx.mean(0))


## Pixel RDE


In [19]:
HPARAMS_PIXEL_RDE['num_steps']=100
# HPARAMS_PIXEL_RDE['l1lambda']=8
# HPARAMS_PIXEL_RDE['step_size']=0.001

In [ ]:
#Pixel RDE
full_array_pixelRDE=torch.zeros((len(files),len(percent_list)))

for i,fname in enumerate(files):
    print(f"Processing file: {fname}")
    x = Image.open(os.path.join(imgdir, fname))
    x = transforms.ToTensor()(x)
    x = transforms.Resize(size=(256,256))(x)
    x = x.to(device).unsqueeze(0)
    
    input_img = Image.open(os.path.join(imgdir, fname))
    input_img = transforms.ToTensor()(input_img)
    input_img = transforms.Resize(size=(256,256))(input_img)
    input_img = input_img.to(device).unsqueeze(0)
    
    # classifiy image
    output = model(x)
    pred = nn.Softmax(dim=1)(output).max(1)[1].item() 
    pixelRDE = PixelRDE(model=model, device=device, **HPARAMS_PIXEL_RDE)
    mask = pixelRDE(x, pred)    
    mask=mask.cpu().numpy()
    with torch.no_grad():
        for j,percent in enumerate(percent_list):
            noise=torch.randn(input_img.shape)
            mask_,masked_img=createmaskedimage(input_img,mask,percent,noise)
            
            if torch.argmax(model(input_img),1)==torch.argmax(model(masked_img.float()),1):

                full_array_pixelRDE[i,j]=1              
    


In [ ]:
np.save('ACpixelrde.npy', full_array_pixelRDE)
# xxx=np.load('ACpixelrde.npy')
# plt.plot(xxx.mean(0))
